In [ ]:
%matplotlib inline
import sys
sys.path.append("..")
from importlib import reload
import numpy as np
from matplotlib import pyplot as plt
import igraph

import pythd

In [ ]:
# Build the dataset
reload(pythd)
reload(pythd.datagen)
datagen = (pythd.datagen.DatasetGenerator()
                .gaussian(center=[-3.0, 0.0], sd=[0.5, 0.5], num_points=200)
                .gaussian(center=[3.0, 0.0], sd=[0.5, 0.5], num_points=200)
                .gaussian(center=[0.0, -3.0], sd=[0.5, 0.5], num_points=200)
                .gaussian(center=[0.0, 3.0], sd=[0.5, 0.5], num_points=200))
dataset = datagen.get()
left_rids = [i for i, p in enumerate(dataset) if p[0] < -3.0]
right_rids = [i for i, p in enumerate(dataset) if p[0] > 3.0]
plt.plot(dataset[:, 0], dataset[:, 1], ".")
plt.show()

In [ ]:
# Setup MAPPER
reload(pythd)
reload(pythd.filter)
reload(pythd.cover)
reload(pythd.clustering)

filt = pythd.filter.IdentityFilter()
f_x = filt(dataset)
cover = pythd.cover.IntervalCover.EvenlySpacedFromValues(f_x, 10, 0.45)
clustering = pythd.clustering.HierarchicalClustering()

In [ ]:
# Run MAPPER
reload(pythd)
reload(pythd.mapper)
reload(pythd.complex)

mapper = pythd.mapper.MAPPER(filter=filt, cover=cover, clustering=clustering)
res = mapper.run(dataset, f_x=f_x)

In [ ]:
# Network visualization
reload(pythd)
reload(pythd.plotting)

res.compute_k_skeleton(k=1)
g = res.get_igraph_network()
layout = g.layout_kamada_kawai()
pythd.plotting.draw_topological_network(res.compute_k_skeleton(k=1), layout)

In [ ]:
c = res.compute_k_skeleton(k=1)

In [ ]:
# Finding connected components in the k-skeleton
components = c.get_connected_components()
groups = []

for nids in components:
    rids = set()
    for nid in nids:
        node = c.get_node((nid,))
        rids = rids | node['points_orig']
    groups.append(rids)

In [ ]:
reload(pythd)
reload(pythd.thd)
reload(pythd.complex)

filt = pythd.filter.IdentityFilter()
f_x = filt(dataset)
cover = pythd.cover.IntervalCover.EvenlySpacedFromValues(f_x, 10, 0.3)
thd = pythd.thd.THD(dataset, filt, cover, group_threshold=100)
thd.run()
root = thd.get_results()

In [ ]:
import igraph
root.color_by_rids(right_rids, normalize=True)
g = root.as_igraph_graph()
vs = {
    "margin": 40,
    "bbox": (350, 350),
    "vertex_label": g.vs["name"],
    "vertex_label_size": 10,
    "vertex_size": 20,
    "vertex_label_dist": 1.5,
    "vertex_label_angle": 0,
    "layout": g.layout_reingold_tilford(root=[0])
}
igraph.plot(g, **vs)

In [ ]:
import json
print(root.get_dict(include_network=False))